In [ ]:
using CUDA
using LinearAlgebra
using MeshGrid

# First define 3D vector as a struct
struct Vec3
    x
    y
    z
end

# introduce Oscillator class: each oscillator has a position and a wavelength
struct Oscillator
    position::Vec3
    lambda::Float64 # wavelength
end

# the main class which really defines all the parameters in the simulation
struct SimulationWorld
    Ncells::Vec3   # amount of cells
    l::Vec3    # cell size
    N_oscillators::Int 
    lambda::Float64  # wavelength
    Phases::Array{Float64}
    Oscillators::Array{Oscillator}
    x::Array{Float64,3} #Define x,y,z coordinate 3D matrices
    y::Array{Float64,3}
    z::Array{Float64,3}
    
    function SimulationWorld(Ncells::Vec3, # inner constructor converts input variables to class properties
            l::Vec3, 
            N_oscillators::Int, 
            lambda::Float64,
            d_phi_x::Float64, # phase difference per oscillator in x-direction
            d_phi_y::Float64, 
            spacing::Float64) # spacing between oscillators
        
        # Make array of oscillators (oscil) and phases 
        oscil = Vector{Oscillator}(undef, N_oscillators)
        phases = zeros(Float64,N_oscillators)
        amount1D = Int(sqrt(N_oscillators))
        for i = 1:N_oscillators
            oscil[i] = Oscillator(Vec3(
                    ((i-1)%amount1D)*spacing  -    (amount1D-1)*spacing/2,
                    (ceil(i/amount1D)-1)*spacing - (amount1D-1)*spacing/2,
                    0), 
                    lambda)
            phases[i] += ((i-1)% amount1D)*d_phi_x + (ceil(i/amount1D)-1)*d_phi_y
        end
        
        # Construct 3D x, y, and z matrices (x-half, y-half, z-half)
        x,y,z = meshgrid( -(Ncells.x*l.x)/2 : l.x : (Ncells.x*l.x)/2,
            -(Ncells.y*l.y)/2 : l.y : (Ncells.y*l.y)/2,
            0 : l.z : Ncells.z*l.z )
        
        new(Ncells, l, N_oscillators, lambda, phases, oscil, x, y, z)
    end
end 

# Use GPU Kernel for quickly evaluating interference over the 3D grid
function calc_Amplitudes!(Amplitudes, nx, ny, nz, x0, y0, z0, x, y, z, k, phase) 
    idx = threadIdx().x + (blockIdx().x - 1) * blockDim().x 
    idy = threadIdx().y + (blockIdx().y - 1) * blockDim().y
    idz = threadIdx().z + (blockIdx().z - 1) * blockDim().z
    if idx <= nx && idy <= ny && idz <= nz 
        
        dist = sqrt( (x[idx,idy,idz] - x0)^2 + (y[idx,idy,idz] - y0)^2 + (z[idx,idy,idz] - z0)^2 )
        dist2 = (x[idx,idy,idz] - x0)^2 + (y[idx,idy,idz] - y0)^2 + (z[idx,idy,idz] - z0)^2
        
        # Calculate the amplitudes and add to previous session
        Amplitudes[idx,idy,idz] += exp(1im*(k*dist + phase))
    end
    return
end

# the GPU kernels are called from this function:
function run(sim::SimulationWorld);
    
    # Define the size of the matrix on GPU
    Amplitudes = CUDA.zeros(ComplexF64, sim.Ncells.x, sim.Ncells.y, sim.Ncells.z); # initialize Amplitudes complex matrix
    
    # breng alle posities naar GPU 
    xgpu = CUDA.CuArray(sim.x); 
    ygpu = CUDA.CuArray(sim.y);
    zgpu = CUDA.CuArray(sim.z);   
    
    # Define the block and grid size
    blocksize = (8, 8, 8);
    gridsizeCO = (
        div(sim.Ncells.x, blocksize[1]) + 1,
        div(sim.Ncells.y, blocksize[2]) + 1,
        div(sim.Ncells.z, blocksize[3]) + 1);
    
    # lanceer kernel, loop over alle oscillators
    for i=1:sim.N_oscillators
        print("Simulation Progress: $(i)%   \r")
        # Do the GPU calculation
        @cuda threads=blocksize blocks=gridsizeCO calc_Amplitudes!(
            Amplitudes, 
            sim.Ncells.x, 
            sim.Ncells.y, 
            sim.Ncells.z, 
            sim.Oscillators[i].position.x, 
            sim.Oscillators[i].position.y, 
            sim.Oscillators[i].position.z, 
            xgpu, # 3D matrices
            ygpu, 
            zgpu, 
            (2*pi)/sim.lambda, # k vector
            sim.Phases[i]); #
        #Amplitudes_cpu = Array(Amplitudes);
    end
    Amp_cpu = Array(Amplitudes)
    return Amp_cpu
end


run (generic function with 1 method)

In [15]:
# prepare the simulation

sim1 = SimulationWorld(
Vec3(128,128,128), # Ncells in each direction
Vec3(0.1,0.1,0.1), # cell size in each direction
25,  # amount of oscillators (only x^2)
1.0, # wavelength
0.0, # steering phase difference in x
0.0, # steering phase difference in y
1.0  # spacing between the oscilators
);

In [ ]:
# Run the simulation:

Waves = run(sim1)

In [ ]:
# plot a cross section of the wave amplitude

using Plots

Amp = abs.(Waves)
crossSection = Amp[:,Int(sim1.Ncells.y/2),:]
heatmap(crossSection, c =:rainbow)

In [ ]:
Amp = abs.(Waves)
crossSection = Amp[Int(sim1.Ncells.x/2),:,:]
heatmap(crossSection, c =:rainbow)

In [ ]:
Wave = real.(Waves)
WaveCrossSec = Wave[Int(sim1.Ncells.x/2),:,:]
heatmap(WaveCrossSec, c =:rainbow)

In [ ]:
# make a 3D plot of the isosurfaces: interactive plot in jupyter notebooks

using WGLMakie                      # for 3D plotting
using Meshing                       # for marching‐cubes / tetrahedra
using Graphs                        # for connectivity analysis
using StaticArrays                  # for small fixed‐size vectors
using GeometryBasics: TriangleFace

WaveNorm = Wave/maximum(Wave); #normalize amplitudes

# 1. Create a test volume
nx, ny, nz = sim1.Ncells.x, sim1.Ncells.y, sim1.Ncells.z
xs = range(-1, 1, length=nx)
ys = range(-1, 1, length=ny)
zs = range(-1, 1, length=nz)
data = copy(WaveNorm)

# 2. Extract both +T and –T isosurfaces
T = 0.25
verts_p, faces_p = isosurface(data, MarchingTetrahedra(iso=T))    # :contentReference[oaicite:0]{index=0}
verts_n, faces_n = isosurface(data, MarchingTetrahedra(iso=-T)) 

# helper: turn a triangle mesh (verts,faces) into components
function split_mesh(verts::Vector{<:NTuple{3,Float64}},
                    faces::Vector{<:NTuple{3,Int}})
    # build connectivity graph
    g = SimpleGraph(length(verts))
    for (i,j,k) in faces
        add_edge!(g, i, j)
        add_edge!(g, j, k)
        add_edge!(g, k, i)
    end
    comps = connected_components(g)

    submeshes = Vector{Tuple{Vector{NTuple{3,Float64}},
                             Vector{NTuple{3,Int}}}}()
    for comp in comps
        # map old→new indices
        remap = Dict(old => new for (new, old) in enumerate(comp))
        pts = verts[comp]
        tris = NTuple{3,Int}[]
        for tri in faces
            # now use haskey, not `in`
            if all(v -> haskey(remap, v), tri)
                push!(tris, (remap[tri[1]],
                             remap[tri[2]],
                             remap[tri[3]]))
            end
        end
        push!(submeshes, (pts, tris))
    end
    return submeshes
end

# split +T and -T
subs_p = split_mesh(verts_p, faces_p)
subs_n = split_mesh(verts_n, faces_n)

# 3. Plot each component
scene = Scene(; size = (800, 800))
cam3d!(scene)

for (pts, tris) in subs_p
    faces = TriangleFace.(tris)
    mesh!(
      scene,
      pts, faces;
      transparency = true,
      color = (:red)
    )
end
for (pts, tris) in subs_n
    faces = TriangleFace.(tris)
    mesh!(
      scene,
      pts, faces;
      transparency = true,
      color = (:blue)
    )
end
scene

In [ ]:
# The following code is a function to save the isosurfaces as an obj, so you can open it in (for example) Blender. 

using GeometryBasics: TriangleFace
using FilePathsBase: basename

"""
    write_colored_obj(objpath, mtlpath,
                      pos_submeshes::Vector,
                      neg_submeshes::Vector)

Write all the `pos_submeshes` and `neg_submeshes` into a single
Wavefront OBJ (`objpath`) plus material file (`mtlpath`).
Positive‐threshold components are colored red; negative ones blue.

Each element of pos_submeshes/neg_submeshes is a `(pts, tris)` tuple:
  • `pts`: `Vector{<:NTuple{3,Real}}`  
  • `tris`: `Vector{<:NTuple{3,Int}}`
"""
function write_colored_obj(objpath::AbstractString,
                           mtlpath::AbstractString,
                           pos_submeshes::Vector,
                           neg_submeshes::Vector)

    # 1) Write the MTL file
    open(mtlpath, "w") do io
        println(io, "# materials for isosurfaces")
        println(io, "newmtl red")
        println(io, "Kd 1.0 0.0 0.0")   # red diffuse
        println(io, "newmtl blue")
        println(io, "Kd 0.0 0.0 1.0")   # blue diffuse
    end

    # 2) Write the OBJ file
    open(objpath, "w") do io
        # Link to the material file
        println(io, "mtllib ", basename(mtlpath))

        vert_offset = 0

        # --- Dump positive meshes (red) ---
        for (i, (pts, tris)) in enumerate(pos_submeshes)
            println(io, "o pos_", i)
            println(io, "usemtl red")
            # vertices
            for (x, y, z) in pts
                println(io, "v ", x, " ", y, " ", z)
            end
            # faces (1-based, add offset)
            for (i1, i2, i3) in tris
                println(io, "f ",
                        i1 + vert_offset, " ",
                        i2 + vert_offset, " ",
                        i3 + vert_offset)
            end
            vert_offset += length(pts)
        end

        # --- Dump negative meshes (blue) ---
        for (i, (pts, tris)) in enumerate(neg_submeshes)
            println(io, "o neg_", i)
            println(io, "usemtl blue")
            for (x, y, z) in pts
                println(io, "v ", x, " ", y, " ", z)
            end
            for (i1, i2, i3) in tris
                println(io, "f ",
                        i1 + vert_offset, " ",
                        i2 + vert_offset, " ",
                        i3 + vert_offset)
            end
            vert_offset += length(pts)
        end
    end

    println("Wrote OBJ → ", objpath)
    println("     MTL → ", mtlpath)
end

In [ ]:
write_colored_obj("ultraPhaseArr_dist_1_labda25os.obj",
                  "ultraPhaseArr_dist_1_labda25os.mtl",
                  subs_p,
                  subs_n)